# http://neuralnetworksanddeeplearning.com/chap1.html

In [94]:
import numpy as np
import random

In [95]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

In [102]:
class Network:
    
    def __init__(self, sizes):
        self.layer_count = len(sizes)
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
        
    def feedforward(self, a):
        for w, b in zip(self.weights, self.biases):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, train_data, epoch, mini_batch_size, lr):
        n = len(train_data)
        for i in xrange(epoch):
            random.shuffle(train_data)
            
            mini_batches = [train_data[k:k+mini_batch_size] for k in xrange(0, n, mini_batch_size)]
            
            for batch in mini_batches:
                self.update(batch, lr)
                
            print("epoch {0} completed".format(i))
            
    def update(self, batch, lr):
        new_b = [np.zeros(b.shape) for b in self.biases]
        new_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in batch:
            delta_b, delta_w = self.backprop(x, y)
            new_b = [tb + db for tb, db in zip(new_b, delta_b)]
            new_w = [tw + dw for tw, dw in zip(new_w, delta_w)]
            
        self.biases = [b - (lr/len(batch))*nb for b, nb in zip(self.biases, new_b)]
        self.weights = [w - (lr/len(batch))*nw for w, nw in zip(self.weights, new_w)]
    
    def backprop(self, x, y):
        nebla_w = [np.zeros(w.shape) for w in self.weights] # weight gradient for cost
        nebla_b = [np.zeros(b.shape) for b in self.biases] # bias gradient for cost
        
        # feedforward
        act = x
        acts = [x] # store all activations, layer-by-layer
        zs = [] # store all z vectors, layer-by-layer
                
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, act) + b
            zs.append(z)
            act = sigmoid(z)
            acts.append(act)
            
        # backward pass
        delta = self.cost_derivative(acts[-1], y) * sigmoid_derivative(zs[-1])
        
        nebla_b[-1] = delta
        nebla_w[-1] = np.dot(delta, acts[-2].T)
        
        for l in xrange(2, self.layer_count):
            z = zs[-l]
            z_derivative = sigmoid_derivative(z)
            delta = np.dot(self.weights[-l+1].T, delta) * z_derivative
            nebla_b[-l] = delta
            nebla_w[-l] = np.dot(delta, acts[-l-1].T)
        return (nebla_b, nebla_w)
    
    def cost_derivative(self, acts, y):
        return (acts - y)
    

In [103]:
import mnist_loader

train_data, val_data, test_data = mnist_loader.load_data_wrapper()

net = Network([784, 30, 10])

net.SGD(train_data, 30, 10, 3.0)

epoch 0 completed
epoch 1 completed
epoch 2 completed
epoch 3 completed


KeyboardInterrupt: 